In [ ]:
try:
    import firedrake
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    import firedrake

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from firedrake import *
import matplotlib.pyplot as plt
import numpy as np

from firedrake.petsc import PETSc

### User defined operators compatible with PETSc KSP solver.
For further info:

https://www.firedrakeproject.org/petsc-interface.html

and the documentation of firedrake and PETSc/petsc4py.

### ***DOCUMENTATION CELL: DO NOT RUN***

In [ ]:
# Object representing a PETSC.Mat M without the actual assembly of M.
# e.g.  M = B*A^{-1}*B' where one does not want to actually compute the inverse A^{-1},
#       but rather solve the system associated to A whenever one needs to compute M*x.
class MatrixFreeOperator(object):

    # Constructor:
    #     - store arguments passed as input
    #     - create all temporary/auxiliary attributes used by method mult.
    def __init__(self, ... ):
        ...
        # e.g. in the case a PETSc.Mat B is passed as input argument:
        self.B = B

        self.tmp1 = B.createVecRight()  # vector to which we can apply B*tmp1
        self.tmp2 = B.createVecLeft()   # vector in which we can store tmp2=B*x
        ...
    
    # Implement y = M*x.
    # Inputs:   mat     - an additional matrix that may be needed (e.g. a preconditioner)
    #                     We will NOT use this argument, which is however needed to agree with KSP.
    #           in_vec  - input_vector x
    # Output:   out_vec - output vector y=M*x
    def mult(self, mat, in_vec, out_vec):
        ...
        # e.g. if M = self.B
        self.B.mult(in_vec, out_vec)
        ...



# Object representing a preconditioner P^{-1} to solve a system by PETSc KSP solver.
# "Similar" to MatrixFreeOperator, but with method apply instead of mult (to agree with KSP)
#   and it needs also a setUp method.
class MatrixFreePC(object):

    # Constructor:
    #     - store arguments passed as input;
    #     - create all temporary/auxiliary attributes used by methods setUp and apply.
    def __init__(self, ...):
        ...

    # Create the preconditioner structure and initialize it.
    def setUp(self, pc):
        ...
        # e.g. if P = M
        self.pc = PETSc.PC().create()
        self.pc.setOptionsPrefix('pc_MatrixFreePC_')
        self.pc.setOperators(self.M)
        self.pc.setFromOptions()
        ...

    # Apply the preconditioner P, namely implement the operation y = P^{-1}*x
    # (actually, solve the system P*y = x).
    # Inputs:   mat     - an additional matrix that may be needed (e.g. a preconditioner)
    #                     We will NOT use this argument, which is however needed to agree with KSP.
    #           in_vec  - input_vector x
    # Output:   out_vec - output vector y=M*x
    def apply(self, mat, in_vec, out_vec):
        ...
        # e.g. if setUp is like in the example above
        self.pc.apply(in_vec, out_vec)
        ...

In [ ]:
# Useful cell to plot ph and uh
fig, ax = plt.subplots()
col = tripcolor(ph, axes=ax)
plt.colorbar(col)
plt.title('pressure')
fig, ax = plt.subplots()
col = quiver(uh, axes=ax)
plt.colorbar(col)
plt.title('velocity')

---
---
# Exercise 1
## Solve steady Stokes problem by Schur-complement method.

### Cavity problem

\begin{equation*}
\begin{cases}
- \Delta \boldsymbol{u} + \nabla  p  = \boldsymbol{0} & {\rm in} \ \Omega=(0,1)^2, \\
{\rm div}\,\boldsymbol{u} = 0 & {\rm in} \ \Omega, \\
\boldsymbol{u} = \boldsymbol{g}_\text{D} & {\rm on} \ \Gamma_4 = (0,1)\times\{4\},\\
\boldsymbol{u} = \boldsymbol{0} & {\rm on} \ \partial\Omega\setminus\Gamma_4,
\end{cases}
\end{equation*}

with $\boldsymbol{g}_\text{D} = 1\boldsymbol{i}$.

For each point of the exercise we create functions, so that it is easier to re-use code.

### Point 1. Assemble the matrices defining the problem (penalty method for Dirichlet BCs).

In [ ]:
def ex1_problem_assembly(n):
    # Mesh, FE spaces (separate, not mixed), and trial and test functions.
    mesh = UnitSquareMesh(n, n, 'crossed')
    V = VectorFunctionSpace(mesh, 'P', 2)
    u = TrialFunction(V)
    v = TestFunction(V)
    Q = FunctionSpace(mesh, 'P', 1)
    p = TrialFunction(Q)
    q = TestFunction(Q)

    # Data.
    g_lid = Constant((1.0,0.0))

    # Variational forms with the penalty method for Dirichlet boundary conditions.
    eps = 1e-20
    a = inner(grad(u), grad(v)) * dx \
        + (1.0/eps)*inner(u,v)*ds           # fully Dirichlet: integral on the whole boundary
    b = - div(u) * q * dx
    L = inner(Constant((0.0,0.0)), v) * dx \
        + (1.0/eps)*inner(g_lid,v)*ds(4)    # Dirichlet conditions are non-homogeneous only on boundary 4

    # Assemble matrices and set-up direct solver for A.
    A_fd = assemble(a)          # firedrake Matrix. No strong bcs: penalty method.
    B = assemble(b).M.handle    # PETSc Mat
    F = assemble(L)
    params = {'ksp_type':'preonly', 'pc_type':'lu'} # direct method
    # params = {'ksp_type':'cg', 'pc_type':'ilu', 'ksp_rtol':1e-3}
    ls_A = LinearSolver(A_fd, solver_parameters=params)
    ksp_A = ls_A.ksp

    return V, Q, ksp_A, B, F

In [ ]:
V, Q, ksp_A, B, F = ex1_problem_assembly(10)

### Point 2. Define classes for Schur complement and M-based preconditioner.

In [ ]:
# Object representing S = B*A^{-1}*B' (see doc above on MatrixFreeOperator).
class SchurComplement(object):
    def __init__(self, ksp_A, B):
        self.ksp_A = ksp_A
        self.B = B
        self.tmp_u1 = B.createVecRight()    # vector to which we can apply B*tmp_u1
        self.tmp_u2 = B.createVecRight()    # vector to which we can apply B*tmp_u2
        self.tmp_p  = B.createVecLeft()     # vector in which we can store tmp_p=B*x
    def mult(self, mat, in_vec, out_vec):
        # Implements out_vec = B*A^{-1}*B'*in_vec

        # tmp_u1 = B'*in_vec
        self.B.multTranspose(in_vec, self.tmp_u1)
        # tmp_u2=A^{-1}*tmp_u1   ->   solve A*tmp_u2 = tmp_u1
        self.ksp_A.solve(self.tmp_u1, self.tmp_u2)
        # out_vec = B*tmp_u2
        self.B.mult(self.tmp_u2, out_vec)

        # print("A solved in", self.ksp_A.getIterationNumber(), "iterations.")


# Create a preconditioner (see doc above on MatrixFreePC)
# based on a PETSC.Mat and possibly using lumping or its diagonal component
class MyMatrixPC(object):

    # Constructor.
    # Input - mat:  the preconditioning matrix
    def __init__(self, mat):
        self.mat = mat

    # Create the preconditioner structure and initialize it.
    def setUp(self, pc):
        S, P_S = pc.getOperators()
        self.pc = PETSc.PC().create()
        self.pc.setOptionsPrefix('pc_MyMatrixPC_')
        self.pc.setOperators(self.mat)
        self.pc.setFromOptions()

    # Implement pc^{-1}*in_vec = out_vec.
    def apply(self, mat, in_vec, out_vec):
        self.pc.apply(in_vec, out_vec)


# Create a preconditioner based on PETSc.Mat P and and set it into ksp.
# Inputs:   ksp solver
#           preconditioner P as PETSc Matrix
def set_KSP_PC(ksp, P):
    MpPC = MyMatrixPC(P)
    pc = ksp.pc
    pc.setType(pc.Type.PYTHON)
    pc.setPythonContext(MpPC)

### Point 3. Solve Stokes cavity problem.

a. Create and setup solvers and preconditioners.

In [ ]:
def ex1_setup_solver(Q, ksp_A, B):
    # Create a PETSc Mat wrapping a SchurComplement instance.
    S = PETSc.Mat().create()
    S.setSizes( B.getSize()[0], B.getSize()[0] )    # B is m-by-n  ->  S is m-by-m (square)
    # print("Size   ->   A:", A_fd.M.handle.getSize(), "  B:", B.getSize(), "  S:", S.getSize())
    S.setType(S.Type.PYTHON)            # i.e. user-defined
    Sctx = SchurComplement(ksp_A, B)    # build the matrix "context" [ https://www.firedrakeproject.org/petsc-interface.html#building-an-operator ]
    S.setPythonContext(Sctx)            # set context into S
    S.setUp()

    # Create KSP solver for system associated to S.
    ksp_S = PETSc.KSP().create()
    ksp_S.setType('cg')     # S is positive definite
    ksp_S.setOperators(S)

    # Create preconditioning matrix and set it into ksp_S.
    p = TrialFunction(Q)
    q = TestFunction(Q)
    Mp = assemble(p*q*dx)
    set_KSP_PC(ksp_S, Mp.M.handle)

    # Set verbose to True for detailed logging of ksp_S.
    # These options are stored GLOBALLY: petsc_options is just a proxy.
    petsc_options = PETSc.Options()
    verbose = False
    if verbose:
        petsc_options['ksp_view'] = ''
        petsc_options['ksp_monitor_true_residual'] = ''
    else:
        del petsc_options['ksp_view']
        del petsc_options['ksp_monitor_true_residual']

    # Finalize setup of ksp_S.
    ksp_S.setFromOptions()
    ksp_S.setUp()

    return ksp_S

In [ ]:
ksp_S = ex1_setup_solver(Q, ksp_A, B)

b. Solve the problem

In [ ]:
def ex1_solve(V, Q, ksp_A, B, ksp_S, vecF):
    # Actual functions to store solution and to create temporary vectors.
    uh = Function(V)
    ph = Function(Q)
    tmp_u_fun = Function(V)
    tmp_p_fun = Function(Q)
    rhs_p_fun = Function(Q)
    # Extract the dof vectors as PETSc.Vec and give them aliases.
    # Temporary vectors are in read/write mode,
    # while momentum rhs is in read-only mode.
    with uh.vector().dat.vec_wo as vecU,\
        ph.vector().dat.vec_wo as vecP,\
        tmp_u_fun.vector().dat.vec_wo as tmp_u,\
        tmp_p_fun.vector().dat.vec_wo as tmp_p,\
        rhs_p_fun.vector().dat.vec_wo as rhs_p:

        # rhs_p = B*(A^{-1}*F)
        ksp_A.solve(vecF, tmp_u)
        B.mult(tmp_u, rhs_p)

        # solve S*P = rhs_p and store it into the dof array of ph
        ksp_S.solve(rhs_p, vecP)

        # reconstruct velocity U = A^{-1}*(F-B'*P) and store it into the dof array of uh
        B.multTranspose(vecP, tmp_u)
        ksp_A.solve(vecF - tmp_u, vecU)

    print("Schur-based solver:")
    print("     # iterations =", ksp_S.getIterationNumber())
    print("     last iter residual norm =", ksp_S.getResidualNorm())
    print("     (only to check convergence of kps_S: it is NOT a measure of the actual error)")

    return uh, ph, mesh

In [ ]:
with F.dat.vec_ro as vecF:
    uh, ph, mesh = ex1_solve(V, Q, ksp_A, B, ksp_S, vecF)

In [ ]:
# Plot ph and uh
fig, ax = plt.subplots()
col = tripcolor(ph, axes=ax)
plt.colorbar(col)
plt.title('pressure')
fig, ax = plt.subplots()
col = quiver(uh, axes=ax)
plt.colorbar(col)
plt.title('velocity')

### Point 4. Compare preconditioners.
Hint: copy-paste cells of point 2 and `ex1_setup_solver` and modify them to implement different preconditioning strategies.
Then use `ex1_solve` as already implemented.

In [ ]:
# Object representing S = B*A^{-1}*B' (see doc above on MatrixFreeOperator).
class SchurComplement(object):
    def __init__(self, ksp_A, B):
        self.ksp_A = ksp_A
        self.B = B
        self.tmp_u1 = B.createVecRight()    # vector to which we can apply B*tmp_u1
        self.tmp_u2 = B.createVecRight()    # vector to which we can apply B*tmp_u2
        self.tmp_p  = B.createVecLeft()     # vector in which we can store tmp_p=B*x
    def mult(self, mat, in_vec, out_vec):
        # Implements out_vec = B*A^{-1}*B'*in_vec

        # tmp_u1 = B'*in_vec
        self.B.multTranspose(in_vec, self.tmp_u1)
        # tmp_u2=A^{-1}*tmp_u1   ->   solve A*tmp_u2 = tmp_u1
        self.ksp_A.solve(self.tmp_u1, self.tmp_u2)
        # out_vec = B*tmp_u2
        self.B.mult(self.tmp_u2, out_vec)

        # print("A solved in", self.ksp_A.getIterationNumber(), "iterations.")


# Create a preconditioner (see doc above on MatrixFreePC)
# based on a PETSC.Mat and possibly using lumping or its diagonal component
class MyMatrixPC(object):
    # Inputs:
    #   - mat:      the preconditioning matrix
    #   - lumping:  if True, use the lumped version of mat
    #   - use_diag: if True, use the diagonal component of mat
    def __init__(self, mat, lumping, use_diag):
        self.mat = mat
        self.lumping = lumping
        self.use_diag = use_diag
        self.vec = self.mat.createVecLeft()

    def setUp(self, pc):
        # check flags
        if self.lumping and self.use_diag:
            raise(BaseException('Error in MyMatrixPC: you can (possibly) either use lumping or diag, not both!'))

        if self.lumping:
            # Compute mat*ones and store it in vec.
            tmp = self.mat.createVecRight()
            tmp.set(1.0)
            self.mat.mult(tmp, self.vec)

        elif self.use_diag:
            self.vec = self.mat.getDiagonal()

        else: # use the full matrix mat
            S, P_S = pc.getOperators()
            self.pc = PETSc.PC().create()
            self.pc.setOptionsPrefix('pc_MyMatrixPC_')
            self.pc.setOperators(self.mat)
            self.pc.setFromOptions()

        # # Print mat and vec. Uncomment only for small mesh.
        #     np.set_printoptions(precision=5)
        #     np.set_printoptions(linewidth=400)
        #     np.set_printoptions(suppress=True)
        # with dim = self.mat.getSizes()[0][0]:
        #     print('Preconditioner: mat =', self.mat.getValues(range(dim), range(dim)))
        # print('vec =', self.vec.getArray())
 
    # Implement "pc^{-1}*in_vec = out_vec" depending on the strategy.
    def apply(self, mat, in_vec, out_vec):
        if self.lumping or self.use_diag:
            # out_vec[i] = in_vec[i]/self.vec[i]
            out_vec.pointwiseDivide(in_vec, self.vec)
        else: # use the full matrix mat
            self.pc.apply(in_vec, out_vec)


# Create a preconditioner based on PETSc.Mat P and and set it into ksp.
# Inputs:   ksp solver
#           preconditioner P as PETSc Matrix
def set_KSP_PC(ksp, P, lumping=False, use_diag=False):
    if lumping and use_diag:
        raise(BaseException('Error in set_KSP_PC: you can (possibly) either use lumping or diag, not both!'))
    MpPC = MyMatrixPC(P, lumping, use_diag)
    pc = ksp.pc
    pc.setType(pc.Type.PYTHON)
    pc.setPythonContext(MpPC)

In [ ]:
def ex1_setup_solver(Q, ksp_A, B):
    # Create a PETSc Mat wrapping a SchurComplement instance.
    S = PETSc.Mat().create()
    S.setSizes( B.getSize()[0], B.getSize()[0] )    # B is m-by-n  ->  S is m-by-m (square)
    # print("Size   ->   A:", A_fd.M.handle.getSize(), "  B:", B.getSize(), "  S:", S.getSize())
    S.setType(S.Type.PYTHON)            # i.e. user-defined
    Sctx = SchurComplement(ksp_A, B)    # build the matrix "context" [ https://www.firedrakeproject.org/petsc-interface.html#building-an-operator ]
    S.setPythonContext(Sctx)            # set context into S
    S.setUp()

    # Create KSP solver for system associated to S.
    ksp_S = PETSc.KSP().create()
    ksp_S.setType('cg')     # S is positive definite
    ksp_S.setOperators(S)

    # Create preconditioner and set it into ksp_S.
    p = TrialFunction(Q)
    q = TestFunction(Q)
    Mp = assemble(p*q*dx)
    # UNCOMMENT TO CHOOSE:
    # pass                                            # no preconditioner: do nothing
    # set_KSP_PC(ksp_S, Mp.M.handle, False, False)    # full Mp matrix
    # set_KSP_PC(ksp_S, Mp.M.handle, True, False)     # lumped Mp
    set_KSP_PC(ksp_S, Mp.M.handle, False, True)     # diag(Mp)

    # Set verbose to True for detailed logging of ksp_S.
    # These options are stored GLOBALLY: petsc_options is just a proxy.
    petsc_options = PETSc.Options()
    verbose = False
    if verbose:
        petsc_options['ksp_view'] = ''
        petsc_options['ksp_monitor_true_residual'] = ''
    else:
        del petsc_options['ksp_view']
        del petsc_options['ksp_monitor_true_residual']

    ksp_S.setFromOptions()  # uses options set above, or default
    ksp_S.setUp()

    return ksp_S

In [ ]:
V, Q, ksp_A, B, F = ex1_problem_assembly(20)
ksp_S = ex1_setup_solver(Q, ksp_A, B)  # ex1_setup_solver has been re-defined: I need to re-call at least this to set up ksp_S
with F.dat.vec_ro as vecF:
    uh, ph, mesh = ex1_solve(V, Q, ksp_A, B, ksp_S, vecF)

# Changing the preconditioner defined in the re-definition of ex1_setup_solver, we notice that
# the number of CG iterations needed to solve the Schur-complement system depends on the preconditioner and it is:
# minimum in the full-Mp case, maximum with no preconditioner, intermediate with the diagonal preconditioners (lumped and diagonal component).

In [ ]:
fig, ax = plt.subplots()
col = tripcolor(ph, axes=ax)
plt.colorbar(col)
plt.title('pressure')
fig, ax = plt.subplots()
col = quiver(uh, axes=ax)
plt.colorbar(col)
plt.title('velocity')

### (additional) Investigate optimality of preconditioners

A preconditioner is **optimal w.r.t. $h$-refinement** if the number of linear solver iterations required to solve the system does not depend on the mesh size $h$.

To better see the differences among the preconditioners, we consider a mesh that allows to better capture the problem's singularity (thus resulting in a more difficult problem to solve). This can be obtained, without increasing the size of the system (i.e. the number of dofs), by redistributing the mesh nodes with a higher concentration near the corners, by means of the function
$$
\vec{f}:\Omega\to\Omega, \qquad f(x,y)=\begin{bmatrix}0.5(1-\cos(\pi x))\\0.5(1-\cos(\pi y))\end{bmatrix},
$$
which maps the domain into itself, but moves points in the desired way (see the mesh plot in the code below).

We investigate the optimality of the preconditioners compared here above by solving our problem on mesh with different refinement levels: $h=0.1\cdot2^{-k}$, for $k=0,1,2,3,4$.

To change the preconditioner, uncomment the corresponding line in the cell of Point4-ex1_setup_solver and run it.


In [ ]:
# We modify the assembly function to introduce the locally refined mesh.
def ex1_problem_assembly_locRefMesh(n):
    # Mesh, FE spaces (separate, not mixed), and trial and test functions.
    mesh = UnitSquareMesh(n, n, 'crossed')

    # Redistribute to have local refinement at corners.
    x = SpatialCoordinate(mesh)
    Vc = mesh.coordinates.function_space()
    f = Function(Vc).interpolate(as_vector([(1-cos(pi*x[0]))/2, (1-cos(pi*x[1]))/2]))
    mesh.coordinates.assign(f)  # evaluates f on the mesh nodes x_i and assigns new nodes positions f(x_i)

    V = VectorFunctionSpace(mesh, 'P', 2)
    u = TrialFunction(V)
    v = TestFunction(V)
    Q = FunctionSpace(mesh, 'P', 1)
    p = TrialFunction(Q)
    q = TestFunction(Q)

    # Data.
    g_lid = Constant((1.0,0.0))

    # Variational forms with the penalty method for Dirichlet boundary conditions.
    eps = 1e-20
    a = inner(grad(u), grad(v)) * dx \
        + (1.0/eps)*inner(u,v)*ds           # fully Dirichlet: integral on the whole boundary
    b = - div(u) * q * dx
    L = inner(Constant((0.0,0.0)), v) * dx \
        + (1.0/eps)*inner(g_lid,v)*ds(4)    # Dirichlet conditions are non-homogeneous only on boundary 4

    # Assemble matrices and set-up direct solver for A.
    A_fd = assemble(a)          # firedrake Matrix. No strong bcs: penalty method.
    B = assemble(b).M.handle    # PETSc Mat
    F = assemble(L)
    params = {'ksp_type':'preonly', 'pc_type':'lu'} # direct method
    # params = {'ksp_type':'cg', 'pc_type':'ilu', 'ksp_rtol':1e-3}
    ls_A = LinearSolver(A_fd, solver_parameters=params)
    ksp_A = ls_A.ksp

    return mesh, V, Q, ksp_A, B, F

In [ ]:
from time import perf_counter
# Solve the problem for different refinement levels.
for n in (10,20,40,80,160):
    print('n =', n, '  -  h =',1.0/n)
    mesh, V, Q, ksp_A, B, F = ex1_problem_assembly_locRefMesh(n)
    fig, ax = plt.subplots()
    triplot(mesh, axes=ax)
    ax.legend()
    ksp_S = ex1_setup_solver(Q, ksp_A, B)  # ex1_setup_solver has been re-defined: I need to re-call at least this to set up ksp_S
    with F.dat.vec_ro as vecF:
        tStart = perf_counter()
        uh, ph, mesh = ex1_solve(V, Q, ksp_A, B, ksp_S, vecF)
        print('elapsed time = ', perf_counter() - tStart, 's    -    # iter = ')


The results are reported in the following table, where we report only some elapsed times, useful for the comparison (NB: the time in seconds could change depending on the system you are running the tests, but the relative comparison still holds):

|h      |   n   |   none    |   Mp      |   lumped  |   diag    |
|-------|-------|-----------|-----------|-----------|-----------|
|0.1    |   10  |   118     |	13      |   17      |   17      |
|0.05   |   20  |   286     |   13      |   18      |	18      |
|0.025  |   40  |   528     |	13	    |   18      |	18      |
|0.0125 |   80  |	928, 9s |	13	    |   18      |	18      |
|0.00625|   160 |	1868    |	13, 7.5s|   18, 7.5s|   18, 7.5s|

In the case of no preconditioner, the number of iterations increases significantly for decreasing values of $h$, whereas all the preconditioners are optimal.
The diagonal preconditioners (lumped, diag) require more linear solver iterations w.r.t. the $M_p$ preconditioner, but the overall cost of the solution in terms of elapsed time is basically the same (for the problem at hand): indeed, "inverting" a diagonal matrix is much less computationally expensive than a non-diagonal one, so the single iteration in the Mp case costs more than in the lumped or diag case.

---
---
# Exercise 2
## Solve unsteady Stokes problem by Schur-complement method and implicit Euler scheme.

### Unsteady cavity problem

\begin{equation*}
\begin{cases}
\frac{\partial\boldsymbol{u}}{\partial t} - \Delta \boldsymbol{u} + \nabla  p  = \boldsymbol{0} & {\rm in} \ \Omega=(0,1)^2
\quad\forall t\in(0,1), \\
{\rm div}\,\boldsymbol{u} = 0 & {\rm in} \ \Omega
\quad\forall t\in(0,1), \\
\boldsymbol{u} = \boldsymbol{g}_\text{D} & {\rm on} \ \Gamma_4 = (0,1)\times\{4\}
\quad\forall t\in(0,1), \\
\boldsymbol{u} = \boldsymbol{0} & {\rm on} \ \partial\Omega\setminus\Gamma_4
\quad\forall t\in(0,1), \\
\boldsymbol{u} = \boldsymbol{0} & {\rm in} \ \Omega,
\quad{\rm for} \ t=0,
\end{cases}
\end{equation*}

with $\boldsymbol{g}_\text{D}(t,\boldsymbol{x}) = t\boldsymbol{i}$.


For the definition of the problem, variational forms, matrices, preconditioners, we use what has been already computed in Exercise 1 - point 4.

In [ ]:
def ex2_problem_assembly(n, dt):
    # Algebraic system at each time step is
    # [ (M+A)*U + B'*P = M*Uold + F
    #   -B*U = 0 ]
    # with A, B, F as in Exercise 1.
    V, Q, ksp_A, B, F = ex1_problem_assembly(n)
    u = TrialFunction(V)
    v = TestFunction(V)
    M = assemble(1/dt * inner(u, v) * dx).M.handle  

    # Define a new KSP solver associated to (M+A).
    A, _ = ksp_A.getOperators()
    ksp_MA = PETSc.KSP().create()
    ksp_MA.setType('preonly')   # direct solver
    ksp_MA.pc.setType('lu')
    ksp_MA.setOperators(M+A)
    ksp_MA.setFromOptions()
    ksp_MA.setUp()

    return V, Q, ksp_MA, M, B, F

In [ ]:
# Time parameters and initial condition.
t0 = 0
T = 1
dt = 0.1
u0 = interpolate(Constant((0., 0.)), V)
print('t =', t0, ' :  ||u||_{L^2} =', norm(u0,'L2'))

# Create the correct Schur complement.
V, Q, ksp_MA, M, B, F = ex2_problem_assembly(20, dt)
ksp_S = ex1_setup_solver(Q, ksp_MA, B)

# Solve time-dependent problem.
tmp_u_fun = Function(V)
rhs_u_fun = Function(V)
with u0.vector().dat.vec_wo as vecUold,\
     F.dat.vec_ro as vecF,\
     tmp_u_fun.vector().dat.vec_wo as tmp_u,\
     rhs_u_fun.vector().dat.vec_wo as rhs:
    for t in np.arange(t0+dt, T, dt):
        # Assemble the current rhs of the system.
        vecF.copy(tmp_u)                # NB NOT tmp_u = vecF (would perform only shallow copy, i.e. reference to same object)
        tmp_u *= t
        M.multAdd(vecUold, tmp_u, rhs)  # rhs = M*vecUold + tmp_u

        # Advance in time: solve the problem with M+A in place of A
        uh, ph, mesh = ex1_solve(V, Q, ksp_MA, B, ksp_S, rhs)
        print('t =', t, ' :  ||u||_{L^2} =', norm(uh,'L2'))
        
        # fig, ax = plt.subplots()
        # col = tripcolor(ph, axes=ax)
        # plt.colorbar(col)
        # plt.title('pressure')
        # fig, ax = plt.subplots()
        # #triplot(mesh, axes=ax)
        # col = quiver(uh, axes=ax)
        # plt.colorbar(col)
        # plt.title('velocity')

        u0 = uh # update the old solution